In [1]:
import json
from transformers import BertTokenizer, BertForSequenceClassification
import torch


In [2]:
def get_medical_recommendations(predicted_label):
    recommendations = {
        "Caries": "Consider using fluoride toothpaste and consult a dentist for further evaluation.",
        "Gingivitis": "Maintain good oral hygiene, including regular brushing and flossing. Consult a dentist for professional cleaning.",
        "Tooth Sensitivity": "Use a toothpaste for sensitive teeth and avoid extreme temperatures. Consult a dentist for personalized advice.",
        "Mouth Ulcer": "Rinse your mouth with saltwater and avoid spicy foods. If persistent, consult a dentist.",
        "Cracked Tooth": "Avoid chewing on hard objects and consult a dentist for possible treatment options.",
        "Cavities": "Maintain good oral hygiene, limit sugary foods, and consult a dentist for cavity treatment.",
        "Tooth Discoloration": "Avoid smoking and limit foods that may stain teeth. Consult a dentist for teeth whitening options.",
        "Hypodontia": "Consult an orthodontist for evaluation and potential treatment options.",
        "Oral Cancer": "Seek immediate medical attention. Consult a healthcare professional for further evaluation.",
        "Unknown": "I'm sorry, I couldn't understand. Please consult a healthcare professional for personalized advice."
    }
    return recommendations.get(predicted_label, "No specific recommendation available.")


In [3]:
class SymptomCheckerBot:
    def __init__(self, dataset,model_save_path="bert_symptom_model"):
        self.dataset = dataset
        self.current_symptom_index = 0
        self.user_responses = {}
        self.no_count = 0  
        self.model_save_path = model_save_path
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

    def start_conversation(self):
        print("Welcome to the AI Clinic! Please answer the following questions with Yes or No.")

        while self.current_symptom_index < len(self.dataset):
            symptom_data = self.dataset[self.current_symptom_index]
            self.ask_question(symptom_data)

        print("Thank you for using the Symptom Checker Bot. Here are the results:")
        self.display_results()

    def ask_question(self, symptom_data):
        for question_data in symptom_data["questions"]:
            question = question_data.get("question")
            label = question_data.get("label")

            if question:
                while True:
                    user_response = input(question + " (Yes/No): ").strip().lower()
                    if user_response in ['yes', 'no']:
                        break
                    else:
                        print("Please enter either 'Yes' or 'No'.")

                self.user_responses[question] = user_response

                if user_response == 'yes':
                    self.no_count = 0
                else:
                    self.no_count += 1

                    if self.no_count >= 2:
                        self.no_count = 0
                        self.change_symptom()  
                        return  

            if label:
                print("\nBased on your responses, you may have:", label)
                if label != "Unknown":
                    classification_result = self.perform_classification(label)
                    print("BERT Classification Result:", classification_result)
                    recommendation = get_medical_recommendations(label)
                    print("Medical Recommendation:", recommendation)

                    self.current_symptom_index = len(self.dataset)  

                self.no_count = 0 
                self.current_symptom_index += 1

    def change_symptom(self):
        self.current_symptom_index += 1
        if self.current_symptom_index >= len(self.dataset):
            print("No more symptoms in the dataset. Exiting.")
            exit()

    def perform_classification(self, text):
        inputs = self.tokenizer(text, return_tensors="pt")
        outputs = self.model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
        return predicted_class

    def display_results(self):
        for question, response in self.user_responses.items():
            print(f"{question}: {response}")
    
    def save_model(self):
        self.tokenizer.save_pretrained(self.model_save_path)
        self.model.save_pretrained(self.model_save_path)
        print(f"Model saved at {self.model_save_path}")

if __name__ == "__main__":
    with open("conversation.json", "r") as file:
        dataset = json.load(file)


In [4]:
bot = SymptomCheckerBot(dataset, model_save_path="chatbot")
bot.start_conversation()
bot.save_model()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Welcome to the AI Clinic! Please answer the following questions with Yes or No.
Is the pain constant or intermittent? (Yes/No): No
Do you feel sensitivity to hot or cold? (Yes/No): No
Do your gums bleed when you brush or floss? (Yes/No): No
Have you noticed any swelling or redness in your gums? (Yes/No): No
Do you feel a sharp pain when consuming hot or cold beverages? (Yes/No): Yes
Does the sensitivity linger after the stimulus is removed? (Yes/No): Yes
Do you have any visible cracks or fractures in your teeth? (Yes/No): Yes
Do you use a hard-bristle toothbrush? (Yes/No): Yes
Have you noticed any receding gums? (Yes/No): Yes
Have you had any recent dental procedures? (Yes/No): Yes
Have you changed your toothpaste recently? (Yes/No): Yes

Based on your responses, you may have: Tooth Sensitivity
BERT Classification Result: 0
Medical Recommendation: Use a toothpaste for sensitive teeth and avoid extreme temperatures. Consult a dentist for personalized advice.
Thank you for using the Symp